In [17]:
import pandas as pd

def read_txt_datasets(show_path, tell_path):
    with open(show_path, 'r') as f:
        show_sents = f.readlines()
    
    with open(tell_path, 'r') as f:
        tell_sents = f.readlines()

    dataset = pd.DataFrame({'sentence': show_sents + tell_sents,
                            'label': ['show'] * len(show_sents) + ['tell'] * len(tell_sents)})

    dataset['sentence'] = dataset['sentence'].str.strip()
    
    return dataset

katia_eval_dataset = read_txt_datasets('katia-show.txt', 'katia-tell.txt')
eval_ds = read_txt_datasets('show-validation.txt', 'tell-validation.txt')
train_ds = pd.read_csv('dataset.csv')


tell    118
show     15
Name: label, dtype: int64

In [3]:
dataset = pd.read_csv('dataset.csv')

In [26]:
dataset = dataset[~dataset['sentence'].isin(katia_dataset['sentence'])]

In [31]:
import csv
dataset.to_csv('dataset.csv', index=False, quoting=csv.QUOTE_ALL)

In [32]:
csv.QUOTE_ALL

1

In [22]:
len(katia_dataset)

133

{'A piece that he had written to clear the agony of his soul had had some far reaching consequence.',
 'A sincere apology is a heart melting and ego diminishing process for both the speaker and the recipient and gives strength to bury the hatchet.',
 'A strenuous workout with the horses under the watchful eyes of his master had taken its toll.',
 'A well branched ancient tree was there, spreading its branches over the wall itself.',
 'After hearing my question her face turned on and she replied with extreme passion.',
 'After the long years of warmth and summer, the cruel winter was approaching.',
 'Alas, words of praise cannot feed empty stomach.',
 'All feared that God would not overlook the slightest sin and would award exemplary punishment to the evil minded person.',
 'All the fingers acted to the occasion as needed and unlocked the door, all he is needs is a just a gentle push to open the door and jump.',
 'All this was drafted in broken sentences ,but was enough to unveil the si

In [15]:
set(katia_dataset['sentence'])

{'A piece that he had written to clear the agony of his soul had had some far reaching consequence.\n',
 'A sincere apology is a heart melting and ego diminishing process for both the speaker and the recipient and gives strength to bury the hatchet.\n',
 'A strenuous workout with the horses under the watchful eyes of his master had taken its toll.\n',
 'A well branched ancient tree was there, spreading its branches over the wall itself.\n',
 'After hearing my question her face turned on and she replied with extreme passion.\n',
 'After the long years of warmth and summer, the cruel winter was approaching.\n',
 'Alas, words of praise cannot feed empty stomach.\n',
 'All feared that God would not overlook the slightest sin and would award exemplary punishment to the evil minded person.\n',
 'All the fingers acted to the occasion as needed and unlocked the door, all he is needs is a just a gentle push to open the door and jump.\n',
 'All this was drafted in broken sentences ,but was enoug

In [8]:
katia_dataset['sentence']

0      The technologist and scientist, the news paper...
1      Ten thousand years ago it was all greenery and...
2      All the fingers acted to the occasion as neede...
3             His eyes forgot to blink and became red.\n
4       The harsh buzzing of the doorbell startled me.\n
                             ...                        
128    The flashing lights now eclipsing our rear vie...
129    The patrol car behind us slammed hard against ...
130    The newly presented diamond ring is much enoug...
131    Like a blinking of eyes two another days passe...
132    It was 6.18 p.m , as i sat sipping the hot bre...
Name: sentence, Length: 133, dtype: object